In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np
import numpy.random as nprand
import pandas as pd
import random
import scipy.stats as spstats
%matplotlib inline

from loomio import *
from socialchoice import *
from netdelib import *

In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = fill_attrition(df_score)

treatments = {
    1: "Single Group",
    2: "Random Pod",
}

df_control = df_score[df_score.treatment == 1]
df_random = df_score[df_score.treatment == 2]

In [ ]:
control_preferences = make_preference_sequence_collection(df_control)
random_preferences = make_preference_sequence_collection(df_random)

In [ ]:
# Note: in this file, stages 0-2 correspond to deliberation pods.
# Stage 3 is the final survey.
df_pods = pd.read_csv('results/pods_2_3.tsv', delimiter='\t')
df_random_pods = df_pods[df_pods.treatment == 2]

In [ ]:
stages = [1,2,3]

all_pods = {}

t = []
delta = []

for stage in stages:
    stage_pods = set()
    pre_stage = stage - 1
    post_stage = stage
    df_stage = df_random_pods[df_random_pods.stage == pre_stage]
    pods = set(df_stage.pod)
    for pod in pods:
        df_pod = df_stage[df_stage.pod == pod]
        pod_participants = frozenset(df_pod.user_id)
        if len(pod_participants) < 2:
            continue
        pre_preferences = [random_preferences[part][pre_stage] for part in pod_participants]
        post_preferences = [random_preferences[part][post_stage] for part in pod_participants]
        pre_profile = Profile(pre_preferences)
        post_profile = Profile(post_preferences)
        d = post_profile.agreement_crossing() - pre_profile.agreement_crossing()
        t.append(stage)
        delta.append(d)
    all_pods[stage] = stage_pods

In [ ]:
plt.figure(figsize=(4,4))
plt.plot(t, delta, '.')
plt.grid()
plt.xticks([1,2,3])
plt.yticks([-2, -1, 0, 1, 2])
plt.xlim([0.5,3.5])
plt.tight_layout()
plt.xlabel('Stage')
plt.ylabel('Change in Pod Agreement (crossing)')